##### Local Environment Setup

In [1]:

import os
import sys
# Set JAVA env variable
os.environ["JAVA_HOME"] = r"C:\Program Files\Eclipse Adoptium\jdk-11.0.26.4-hotspot"
# Set Hadoop environment variables 
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = os.environ['HADOOP_HOME'] + r'\bin;' + os.environ['PATH']
# Set the Python executable path explicitly
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Get Pipeline Tools Path
sys.path.append(R'C:\GitHub\Tools\de')


##### Libraries

In [2]:
import time
import logging
from datetime import datetime

import numpy as np
import pandas as pd

from typing import Dict


from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import (StructType, StructField, StringType, 
                            DoubleType, IntegerType, TimestampType, 
                            DateType)
from delta.tables import DeltaTable


In [3]:
from de_pipeline_tools import *

##### Spark Session

In [4]:
spark = initialize_local_spark_delta_lake("Financial Data Pipeline")

2025-04-24 15:54:16,728 - INFO - ---Spark session initialized with Delta Lake support---


In [5]:
# Create a database in the Hive warehouse if doesn't exist
spark.sql("CREATE DATABASE IF NOT EXISTS de_pipelines LOCATION 'C:/hive-warehouse/de_pipelines'")

DataFrame[]


## SCD Type 2:
##### Intial Creation (Overwrite)

##### Inputs/Outputs

In [6]:
# Intial Creation
data_dir = "../../data"
file_list = [
    "financial_transactions_20250409_113413.csv",
    "financial_transactions_20250410_113413.csv"
]

abs_file_list = []
for file in file_list:
    # Join data_dir with filename
    relative_path = os.path.join(data_dir, file)
    # Convert to absolute path
    abs_path = os.path.abspath(relative_path)
    abs_file_list.append(abs_path)

bronze_table = 'de_pipelines.financial_osb_bronze_type2'
silver_table = 'de_pipelines.financial_osb_silver_type2'
gold_table   = 'de_pipelines.financial_osb_gold_type2'


In [7]:
def get_schema():
    return StructType([
    StructField("transaction_id", StringType(), False),
    StructField("timestamp", StringType(), True), #load in as string type, handle in silver step     
    StructField("customer_id", StringType(), True),      
    StructField("account_number", StringType(), True),   
    StructField("transaction_type", StringType(), True), 
    StructField("amount", DoubleType(), True),           
    StructField("currency", StringType(), True),         
    StructField("balance_after", DoubleType(), True),    
    StructField("status", StringType(), True),           
    StructField("merchant", StringType(), True),         
    StructField("category", StringType(), True),         
    StructField("location", StringType(), True)          
])

#### Define: Validation Rules, Transformations, Write Executions

In [8]:
def bronze_transform(df:DataFrame) -> DataFrame:

    # Mark the most recent record as current
    # and the rest as historical
    w = Window.partitionBy("transaction_id", "account_number").orderBy(F.col("timestamp").desc())

    # define SCD Type 2 necessary inputs
    df = df.withColumn("row_num", F.row_number().over(w)) \
           .withColumn("is_current", F.when(F.col("row_num") == 1, 1).otherwise(0)) \
           .withColumn("start_date", F.current_date()) \
           .withColumn("end_date", F.when(F.col("is_current") == 0, F.current_date()).otherwise(F.lit(None).cast("timestamp"))) \
           .drop("row_num")
           
    return df

In [9]:
def bronze_writer(df: DataFrame, table_name, bronze_transform=None):

    """Write data to bronze layer with optional transformation"""
    try:
        if bronze_transform:
            logger.info("Applying transformation within bronze writer")
            transformed_df = bronze_transform(df)
        else:
            transformed_df = df
            
        (transformed_df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(table_name)
        )
        
    except Exception as e:
        logger.error(f"Error in bronze_writer: {str(e)}")
        raise
    

    
    return None

In [10]:
# Define bronze validation rules
bronze_validation_rules = [
    {
        "name": "has_transaction_id",
        "condition": "transaction_id IS NOT NULL",
        "description": "Transaction ID must be present"
    },
    {
        "name": "valid_amount",
        "condition": "amount IS NOT NULL AND amount > 0",
        "description": "Amount must be positive if not null"
    },
    {
        "name": "valid_timestamp",
        "condition": "timestamp IS NOT NULL AND timestamp <= current_timestamp()",
        "description": "Timestamp must not be in the future"
    }
]

In [11]:
def silver_transform(df:DataFrame) -> DataFrame:

    # Clean any non-timestamp characters first
    df = df.withColumn(
        "timestamp", 
        F.regexp_replace(F.col("timestamp"), "[^0-9\\-: ]", "")
    )

    # Cast to timestamp type
    df = df.withColumn("timestamp", F.col("timestamp").cast("timestamp"))

    # Remove duplicates
    df = df.dropDuplicates()

    # Standardize Data
    df = (df
            .withColumn("amount", F.abs(F.col("amount")))
            .withColumn("transaction_type", F.lower(F.col("transaction_type")))
            .withColumn("category", F.lower(F.col("category")))
            .withColumn("status", F.lower(F.col("status")))
    )

    # Filter Data
    # Address bronze layer data validation check concerns
    df = df.filter(
                    (F.col('transaction_id').isNotNull()) # transaction id must exist
                    & (F.col('account_number').isNotNull()) # account number must exist
                    & (F.col('amount') > 0) # amount must be positive
                    & ((F.col('timestamp') <= F.current_date()) # must be <= current date
                    |(F.col('timestamp').isNull()))# or must be Null, no future timestamps
    ) 
    
    # Split timestamp into date and time and year_month for paritioning
    df = (df
            .withColumn("transaction_date", F.to_date("timestamp"))
            .withColumn("transaction_time", F.date_format("timestamp", "HH:mm:ss"))
            .withColumn("year_month", F.date_format(F.col("transaction_date"), "yyyy-MM"))
    )
    
    # Add processing timestamp for bookkeeping
    df = (df
            .withColumn("processing_timestamp", F.current_timestamp())
    )

    return df

In [12]:
def silver_writer(df: DataFrame, table_name, silver_transform=None):

    """Write data to silver layer with optional transformation"""
    try:
        if silver_transform:
            logger.info("Applying transformation within silver writer")
            transformed_df = silver_transform(df)
        else:
            transformed_df = df
            
        (transformed_df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .partitionBy("year_month")
        .saveAsTable(table_name)
        )

    except Exception as e:
        logger.error(f"Error in silver_writer: {str(e)}")
        raise


    return None

In [13]:
# Define silver validation rules
silver_validation_rules = [
    {
        "name": "valid_transaction_type",
        "condition": "transaction_type IN ('debit', 'credit', 'transfer', 'payment', 'withdrawal', 'deposit') OR transaction_type IS NULL",
        "description": "Transaction type must be one of the valid types"
    },
    {
        "name": "valid_status",
        "condition": "status IN ('completed', 'pending', 'failed', 'cancelled', 'refunded') OR status IS NULL",
        "description": "Status must be one of the valid statuses"
    },
    {
        "name": "valid_currency",
        "condition": "currency IS NOT NULL AND length(currency) = 3",
        "description": "Currency code should be 3 characters if present"
    },
    {
        "name": "valid_timestamp",
        "condition": "timestamp IS NOT NULL AND timestamp <= current_timestamp()",
        "description": "Timestamp must not be NULL or in the future"
    }
]

In [14]:
def gold_transform(df:DataFrame) -> Dict:
    gold_dfs = {}

    # Gold aggregation 1: Daily summary by category
    daily_category = (df
        .groupBy("transaction_date", "category")
        .agg(
            F.count("transaction_id").alias("transaction_count"),
            F.sum("amount").alias("total_amount"),
            F.avg("amount").alias("avg_amount"),
            F.min("amount").alias("min_amount"),
            F.max("amount").alias("max_amount"),
            F.countDistinct("customer_id").alias("unique_customers")
        )
        .withColumn("processing_timestamp", F.current_timestamp())
    )
    
    gold_dfs["daily_category"] = daily_category
    
    # Gold aggregation 2: Customer summary
    customer_summary = (df
        .groupBy("customer_id")
        .agg(
            F.count("transaction_id").alias("transaction_count"),
            F.sum("amount").alias("total_amount"),
            F.avg("amount").alias("avg_amount"),
            F.min("transaction_date").alias("first_transaction_date"),
            F.max("transaction_date").alias("last_transaction_date"),
            F.approx_count_distinct("category").alias("category_count")
        )
        .withColumn("processing_timestamp", F.current_timestamp())
        .withColumn("days_since_last_transaction", 
                    F.datediff(F.current_date(), F.col("last_transaction_date")))
    )
    
    gold_dfs["customer_summary"] = customer_summary
    
    # Gold aggregation 3: Transaction type summary
    transaction_type_summary = (df
        .groupBy("transaction_type")
        .agg(
            F.count("transaction_id").alias("transaction_count"),
            F.sum("amount").alias("total_amount"),
            F.avg("amount").alias("avg_amount")
        )
        .withColumn("processing_timestamp", F.current_timestamp())
    )
    
    gold_dfs["transaction_type_summary"] = transaction_type_summary

    return gold_dfs

In [15]:
def gold_writer(df: DataFrame, table_name: str):
    (df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(table_name)
    )

    return None

In [16]:
# Define gold validation rules
gold_validation_rules = [
    {
        "name": "positive_transaction_counts",
        "condition": "transaction_count > 0",
        "description": "Transaction counts should be positive"
    },
    {
        "name": "valid_total_amounts",
        "condition": "total_amount >= 0",
        "description": "Total amounts should not be negative"
    }
]

### Running Full Batch Pipeline

In [17]:
run_batch_de_pipeline(spark, 'csv', abs_file_list, get_schema(), 
                     bronze_table=bronze_table, silver_table=silver_table, gold_table=gold_table, 
                     bronze_transform=bronze_transform, silver_transform=silver_transform, gold_transform=gold_transform,
                     bronze_writer=bronze_writer, silver_writer=silver_writer, gold_writer=gold_writer,
                     bronze_validation_rules=bronze_validation_rules, 
                     silver_validation_rules=silver_validation_rules, 
                     gold_validation_rules=gold_validation_rules,
                     pipeline_name='Financial_DE_Pipeline')

2025-04-24 15:54:42,108 - INFO - --Starting data pipeline execution with ID: Financial_DE_Pipeline_20250424_155442--
2025-04-24 15:54:42,110 - INFO - Starting bronze layer processing
2025-04-24 15:54:43,016 - INFO - Successfully read CSV data from: 
  - c:\GitHub\DE_Pipelines\data\financial_transactions_20250409_113413.csv
  - c:\GitHub\DE_Pipelines\data\financial_transactions_20250410_113413.csv
2025-04-24 15:54:43,018 - INFO - Writing to bronze table: de_pipelines.financial_osb_bronze_type2
2025-04-24 15:54:45,465 - WARNING - Table de_pipelines.financial_osb_bronze_type2 not found
2025-04-24 15:54:45,467 - INFO - Applying transformation within bronze writer
2025-04-24 15:55:12,267 - INFO - Successfully wrote to bronze table: de_pipelines.financial_osb_bronze_type2
2025-04-24 15:55:12,269 - INFO - Write Metrics: 
[
  {
    "numOutputRows": "1000",
    "numOutputBytes": "43254",
    "numFiles": "1"
  }
]
2025-04-24 15:55:13,097 - INFO - Running data quality checks for bronze layer
2025

{'status': 'success',
 'pipeline_id': 'Financial_DE_Pipeline_20250424_155442',
 'bronze_version': 0,
 'silver_version': 0,
 'timestamp': '2025-04-24T15:57:32.796488',
 'duration_seconds': 170.68695998191833,
 'metrics': {'pipeline_id': 'Financial_DE_Pipeline_20250424_155442',
  'start_time': '2025-04-24T15:54:42.108527',
  'stages': {'bronze': {'duration_seconds': 52.77463984489441,
    'version': 0,
    'status': 'success'},
   'bronze_optimize': {'layer': 'bronze',
    'duration_seconds': 3.885894298553467,
    'status': 'success'},
   'silver': {'duration_seconds': 32.11255216598511,
    'version': 0,
    'status': 'success',
    'source_bronze_version': 0},
   'silver_optimize': {'layer': 'silver',
    'duration_seconds': 12.202362537384033,
    'status': 'success'},
   'gold': {'duration_seconds': 63.91578125953674,
    'status': 'success',
    'source_silver_version': 0,
    'tables': ['daily_category',
     'customer_summary',
     'transaction_type_summary']},
   'gold_optimize

### Running Layers In Isolation

In [18]:
spark = initialize_local_spark_delta_lake("Financial Data Pipeline - Testing")

2025-04-24 15:57:34,374 - INFO - ---Spark session initialized with Delta Lake support---


#### Bronze

In [19]:
bronzedf, bronze_version = process_batch_bronze_layer(spark, 'csv', abs_file_list, get_schema(), bronze_table,
                               bronze_transform=bronze_transform, validation_rules=bronze_validation_rules,
                               pipeline_id='test', mode='test', bronze_writer=None)

2025-04-24 15:57:35,850 - INFO - Starting bronze layer processing
2025-04-24 15:57:35,910 - INFO - Successfully read CSV data from: 
  - c:\GitHub\DE_Pipelines\data\financial_transactions_20250409_113413.csv
  - c:\GitHub\DE_Pipelines\data\financial_transactions_20250410_113413.csv
2025-04-24 15:57:35,911 - WARNING - --- Bronze layer in Test Mode ---
2025-04-24 15:57:35,984 - INFO - Transformation function applied in test mode
2025-04-24 15:57:35,986 - INFO - Running data quality checks for bronze layer
2025-04-24 15:57:42,983 - INFO - Data Quality Metrics for bronze layer:
2025-04-24 15:57:42,984 - INFO -   - Shape: [18,1000] (approx. row count)
2025-04-24 15:57:42,985 - INFO -   - Schema: 
root
 |-- transaction_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- currency: string (nullable 

In [20]:
bronzedf.show(5)

+--------------+-------------------+-----------+--------------+----------------+-------+--------+-------------+--------+--------------------+---------+---------------+--------------------+--------------------+--------+
|transaction_id|          timestamp|customer_id|account_number|transaction_type| amount|currency|balance_after|  status|            merchant| category|       location| ingestion_timestamp|         source_file|batch_id|
+--------------+-------------------+-----------+--------------+----------------+-------+--------+-------------+--------+--------------------+---------+---------------+--------------------+--------------------+--------+
|   TXN00000001|2023-05-05 03:42:36| CUST001179| ACCT-39450273|        interest|1378.52|     USD|      9544.56|reversed|                NULL|     NULL|Los Angeles, CA|2025-04-24 15:57:...|file:///c:/GitHub...|    test|
|   TXN00000002|2023-08-11 06:24:46| CUST001188| ACCT-55298556|          refund| 1626.4|     JPY|      9598.17|disputed|    

#### Silver

In [21]:
silverdf, silver_version = process_batch_silver_layer(spark, bronze_table, bronze_version=None, 
                                                   silver_table=silver_table, 
                                                   silver_transform=silver_transform, 
                                                   validation_rules=silver_validation_rules,
                                                   pipeline_id='test', mode='test', 
                                                   silver_writer=None)

2025-04-24 15:57:50,619 - INFO - Starting silver layer processing
2025-04-24 15:57:51,309 - INFO - Successfully read bronze data version 0
2025-04-24 15:57:51,311 - WARNING - --- Silver layer in Test Mode ---
2025-04-24 15:57:51,451 - INFO - Transformation function applied in test mode
2025-04-24 15:57:51,453 - INFO - Running data quality checks for silver layer
2025-04-24 15:58:01,347 - INFO - Data Quality Metrics for silver layer:
2025-04-24 15:58:01,349 - INFO -   - Shape: [22,950] (approx. row count)
2025-04-24 15:58:01,350 - INFO -   - Schema: 
root
 |-- transaction_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- balance_after: double (nullable = true)
 |-- status: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- catego

In [22]:
silverdf.show(5)

+--------------+-------------------+-----------+--------------+----------------+-------+--------+-------------+---------+--------+--------+-----------------+--------------------+--------------------+--------------------+----------+----------+--------+
|transaction_id|          timestamp|customer_id|account_number|transaction_type| amount|currency|balance_after|   status|merchant|category|         location| ingestion_timestamp|         source_file|            batch_id|is_current|start_date|end_date|
+--------------+-------------------+-----------+--------------+----------------+-------+--------+-------------+---------+--------+--------+-----------------+--------------------+--------------------+--------------------+----------+----------+--------+
|          NULL|2023-09-11 10:07:08| CUST001016| ACCT-19335534|          refund|1248.85|     CAD|     12133.35|completed|   Zelle| savings|       Denver, CO|2025-04-24 15:54:...|file:///c:/GitHub...|Financial_DE_Pipe...|         1|2025-04-24|  

##### Investigating Silver Validation Check Failures:
Corrupt "status" column

In [23]:
silverdf.groupBy('status').count().show()

+---------+-----+
|   status|count|
+---------+-----+
| dqsputed|    1|
|  pgnding|    1|
|completed|  206|
|   failen|    1|
|   failed|  186|
|bompleted|    1|
|     NULL|   19|
| reyersed|    1|
|comcleted|    1|
| disputad|    1|
| dmsputed|    1|
| diiputed|    1|
|  pending|  171|
| disputed|  204|
| reverszd|    1|
| reversed|  204|
+---------+-----+



#### Gold

In [24]:
gold_dfs = process_batch_gold_layer(spark, silver_table, silver_version=None, 
                                 gold_table=gold_table, 
                                 gold_transform=gold_transform, 
                                 validation_rules=gold_validation_rules, 
                                 pipeline_id='test', 
                                 mode='test',
                                 gold_writer=None)

2025-04-24 15:58:12,646 - INFO - Starting gold layer processing
2025-04-24 15:58:13,073 - INFO - Successfully read silver data version 1
2025-04-24 15:58:13,205 - INFO - Transformation function applied
2025-04-24 15:58:13,206 - WARNING - Gold layer in Test Mode
2025-04-24 15:58:13,207 - INFO - Running data quality checks for de_pipelines.financial_osb_gold_type2_daily_category layer
2025-04-24 15:58:23,881 - INFO - Data Quality Metrics for de_pipelines.financial_osb_gold_type2_daily_category layer:
2025-04-24 15:58:23,882 - INFO -   - Shape: [9,702] (approx. row count)
2025-04-24 15:58:23,884 - INFO -   - Schema: 
root
 |-- transaction_date: date (nullable = true)
 |-- category: string (nullable = true)
 |-- transaction_count: long (nullable = false)
 |-- total_amount: double (nullable = true)
 |-- avg_amount: double (nullable = true)
 |-- min_amount: double (nullable = true)
 |-- max_amount: double (nullable = true)
 |-- unique_customers: long (nullable = false)
 |-- processing_timest

In [25]:
for table_name in gold_dfs.keys():
    gold_dfs[table_name].show(5)

+----------------+----------+-----------------+------------+----------+----------+----------+----------------+--------------------+
|transaction_date|  category|transaction_count|total_amount|avg_amount|min_amount|max_amount|unique_customers|processing_timestamp|
+----------------+----------+-----------------+------------+----------+----------+----------+----------------+--------------------+
|      2023-05-25|      fees|                1|       30.89|     30.89|     30.89|     30.89|               1|2025-04-24 15:58:...|
|      2023-09-25|    income|                1|      1384.1|    1384.1|    1384.1|    1384.1|               1|2025-04-24 15:58:...|
|      2023-06-22|      NULL|                1|       64.69|     64.69|     64.69|     64.69|               1|2025-04-24 15:58:...|
|      2023-12-18|investment|                1|      626.53|    626.53|    626.53|    626.53|               1|2025-04-24 15:58:...|
|      2023-08-28|   housing|                1|       37.49|     37.49|     


## SCD Type 2: 
##### Change Data Capture (Merge)

##### Inputs/Outputs

In [26]:
# Change Data Capture
cdc_file_list = [
    "financial_transactions_20250410_113413_update.csv",
    "financial_transactions_20250414_113413.csv",
    "financial_transactions_20250417_113413.csv"
]

cdc_abs_file_list = []
for file in cdc_file_list:
    # Join data_dir with filename
    relative_path = os.path.join(data_dir, file)
    # Convert to absolute path
    abs_path = os.path.abspath(relative_path)
    cdc_abs_file_list.append(abs_path)


#### Define: Validation Rules(Same), Transformations(Same), Write Executions (Merge)

In [27]:
def cdc_bronze_writer(df: DataFrame, table_name: str, bronze_transform=None):

    # Apply Pre-Transformation Filters:

    # Split into latest and historical records
    df = df.withColumn("row_num", F.row_number().over(
        Window.partitionBy("transaction_id", "account_number")
        .orderBy(F.col("timestamp").desc())
        )
    )
    archivedf = df.filter(F.col("row_num") != 1).drop("row_num")
    df = df.filter(F.col("row_num") == 1).drop("row_num")


    """Write data to bronze layer with optional transformation"""
    try:
        if bronze_transform:
            logger.info("Applying transformation within bronze writer")
            transformed_df = bronze_transform(df)
            transformed_archivedf = bronze_transform(archivedf)
        else:
            transformed_df = df
            transformed_archivedf = archivedf
            
        
        transformed_df.createOrReplaceTempView("source")

        # Update existing records
        spark.sql(f"""
            MERGE INTO {table_name} t
            USING source s
            ON t.transaction_id = s.transaction_id
                AND t.account_number = s.account_number
                AND t.is_current=1
                AND t.balance_after != s.balance_after
            WHEN MATCHED THEN
            UPDATE SET
                t.is_current = 0,
                t.end_date = CURRENT_DATE()
        """)

        # Insert new records
        spark.sql(f"""
            MERGE INTO {table_name} t
            USING (
                SELECT
                    *,
                    1 AS is_current,
                    CURRENT_DATE() AS start_date,
                    CAST(NULL AS TIMESTAMP) AS end_date
                FROM source
            ) s
            ON s.transaction_id = t.transaction_id 
                AND s.account_number = t.account_number 
                AND t.is_current=1
            WHEN NOT MATCHED THEN
                INSERT *
        """)

        # Handle historical records
        if not transformed_archivedf.rdd.isEmpty():
            transformed_archivedf.createOrReplaceTempView("archive")

            # Insert Historical Records using MERGE
            spark.sql(f"""
                MERGE INTO {table_name} t
                USING (
                    SELECT
                        a.*,
                        0 AS is_current,
                        CURRENT_DATE() AS start_date,
                        CURRENT_DATE() AS end_date
                    FROM archive a
                    INNER JOIN {table_name} t2
                    ON a.transaction_id = t2.transaction_id 
                        AND a.account_number = t2.account_number
                    WHERE t2.is_current = 1
                    AND NOT EXISTS (
                        SELECT 1 FROM {table_name} t3
                        WHERE t3.transaction_id = a.transaction_id
                        AND t3.account_number = a.account_number
                        AND t3.balance_after = a.balance_after
                        AND t3.is_current = 0
                    )
                ) AS matched_records
                ON 1=0  -- Forces all records to INSERT (if there are any)
                WHEN NOT MATCHED THEN
                    INSERT *
            """)
    
    except Exception as e:
        logger.error(f"Error in bronze_writer: {str(e)}")
        raise

    return None

In [28]:
def cdc_silver_writer(df: DataFrame, table_name, silver_transform=None):

    # Apply Pre-Transformation Filters:

    # get max ingested timestamp from silver table
    max_ts = spark.sql(f"SELECT MAX(ingestion_timestamp) FROM {table_name}").collect()[0][0]

    # filter incoming data to only include records with a later ingested timestamp
    df = df.filter(F.col("ingestion_timestamp") > max_ts)


    """Write data to silver layer with optional transformation"""
    try:
        if silver_transform:
            logger.info("Applying transformation within silver writer")
            transformed_df = silver_transform(df)
        else:
            transformed_df = df

        transformed_df.createOrReplaceTempView("source")

        # Update Existing Records
        spark.sql(f"""
            MERGE INTO {table_name} t
            USING source s
            ON t.transaction_id = s.transaction_id
                AND t.account_number = s.account_number
                AND t.is_current=1
                AND s.is_current=1
                AND t.balance_after != s.balance_after
            WHEN MATCHED THEN
            UPDATE SET
                t.is_current = 0,
                t.end_date = CURRENT_DATE()

        """)

        # Insert New Records
        spark.sql(f"""
            MERGE INTO {table_name} t
            USING source s
            ON s.transaction_id = t.transaction_id 
                AND s.account_number = t.account_number 
                AND t.is_current=1
            WHEN NOT MATCHED AND s.is_current=1 THEN
                INSERT *
        """)

        # Insert Historical Records
        spark.sql(f"""
            MERGE INTO {table_name} t
            USING (
                SELECT s.*
                FROM source s
                INNER JOIN {table_name} t2
                ON s.transaction_id = t2.transaction_id 
                    AND s.account_number = t2.account_number
                WHERE t2.is_current = 1
                    AND s.is_current = 0
                    AND NOT EXISTS (
                        SELECT 1 FROM {table_name} t3
                        WHERE t3.transaction_id = s.transaction_id
                        AND t3.account_number = s.account_number
                        AND t3.balance_after = s.balance_after
                        AND t3.is_current = 0
                    )
            ) AS matched_records
            ON 1=0  -- This ensures all records go to the INSERT clause -- always false
            WHEN NOT MATCHED THEN
            INSERT *
        """)

    except Exception as e:
        logger.error(f"Error in silver_writer: {str(e)}")
        raise

    return None

#### Running CDC Full Batch Pipeline (Bronze, Silver)

In [29]:
spark = initialize_local_spark_delta_lake("Financial Data Pipeline_CDC")

2025-04-24 15:59:03,299 - INFO - ---Spark session initialized with Delta Lake support---


In [30]:
run_batch_de_pipeline(spark, 'csv', cdc_abs_file_list, get_schema(), 
                     bronze_table=bronze_table, silver_table=silver_table, gold_table=None, 
                     bronze_transform=None, silver_transform=silver_transform, gold_transform=None,
                     bronze_writer=cdc_bronze_writer, silver_writer=cdc_silver_writer, gold_writer=None,
                     bronze_validation_rules=bronze_validation_rules, 
                     silver_validation_rules=silver_validation_rules, 
                     gold_validation_rules=None,
                     pipeline_name='Financial_DE_Pipeline_CDC')

2025-04-24 15:59:03,890 - INFO - --Starting data pipeline execution with ID: Financial_DE_Pipeline_CDC_20250424_155903--
2025-04-24 15:59:03,894 - INFO - Starting bronze layer processing
2025-04-24 15:59:03,970 - INFO - Successfully read CSV data from: 
  - c:\GitHub\DE_Pipelines\data\financial_transactions_20250410_113413_update.csv
  - c:\GitHub\DE_Pipelines\data\financial_transactions_20250414_113413.csv
  - c:\GitHub\DE_Pipelines\data\financial_transactions_20250417_113413.csv
2025-04-24 15:59:03,972 - INFO - Writing to bronze table: de_pipelines.financial_osb_bronze_type2
2025-04-24 15:59:27,503 - INFO - Successfully wrote to bronze table: de_pipelines.financial_osb_bronze_type2
2025-04-24 15:59:27,504 - INFO - Write Metrics: 
[
  {
    "numOutputRows": "1000",
    "numTargetBytesAdded": "43685",
    "numTargetRowsInserted": "0",
    "numTargetFilesAdded": "1",
    "materializeSourceTimeMs": "1005",
    "numTargetRowsMatchedDeleted": "0",
    "numTargetFilesRemoved": "1",
    "num

{'status': 'success',
 'pipeline_id': 'Financial_DE_Pipeline_CDC_20250424_155903',
 'bronze_version': 3,
 'silver_version': 4,
 'timestamp': '2025-04-24T16:00:29.544475',
 'duration_seconds': 85.65281677246094,
 'metrics': {'pipeline_id': 'Financial_DE_Pipeline_CDC_20250424_155903',
  'start_time': '2025-04-24T15:59:03.890658',
  'stages': {'bronze': {'duration_seconds': 30.126533269882202,
    'version': 3,
    'status': 'success'},
   'bronze_optimize': {'layer': 'bronze',
    'duration_seconds': 6.246767520904541,
    'status': 'success'},
   'silver': {'duration_seconds': 35.55402660369873,
    'version': 4,
    'status': 'success',
    'source_bronze_version': 3},
   'silver_optimize': {'layer': 'silver',
    'duration_seconds': 13.717478275299072,
    'status': 'success'}},
  'status': 'success',
  'end_time': '2025-04-24T16:00:29.543475',
  'total_duration_seconds': 85.65281677246094}}

#### CDC Validation

In [31]:
spark = initialize_local_spark_delta_lake("Financial Data Pipeline - Testing")

2025-04-24 16:00:30,478 - INFO - ---Spark session initialized with Delta Lake support---


In [32]:
spark.read.table(bronze_table).filter(F.col("transaction_id") == "TXN00000501").show()

+--------------+-------------------+-----------+--------------+----------------+--------+--------+-------------+--------+--------+--------+------------+--------------------+--------------------+--------------------+----------+----------+-------------------+
|transaction_id|          timestamp|customer_id|account_number|transaction_type|  amount|currency|balance_after|  status|merchant|category|    location| ingestion_timestamp|         source_file|            batch_id|is_current|start_date|           end_date|
+--------------+-------------------+-----------+--------------+----------------+--------+--------+-------------+--------+--------+--------+------------+--------------------+--------------------+--------------------+----------+----------+-------------------+
|   TXN00000501|2023-01-24 22:00:49| CUST001103| ACCT-56020613|             fee|  -89.25|     USD|     333333.0|disputed|   Chase|    fees|New York, NY|2025-04-24 15:59:...|file:///c:/GitHub...|Financial_DE_Pipe...|         0|

In [33]:
spark.read.table(silver_table).filter(F.col("transaction_id") == "TXN00000501").show()

+--------------+-------------------+-----------+--------------+----------------+--------+--------+-------------+--------+--------+--------+------------+--------------------+--------------------+--------------------+----------+----------+-------------------+----------------+----------------+----------+--------------------+
|transaction_id|          timestamp|customer_id|account_number|transaction_type|  amount|currency|balance_after|  status|merchant|category|    location| ingestion_timestamp|         source_file|            batch_id|is_current|start_date|           end_date|transaction_date|transaction_time|year_month|processing_timestamp|
+--------------+-------------------+-----------+--------------+----------------+--------+--------+-------------+--------+--------+--------+------------+--------------------+--------------------+--------------------+----------+----------+-------------------+----------------+----------------+----------+--------------------+
|   TXN00000501|2023-01-24 2